In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import keras
import cv2
import tensorflow as tf
x_train=[]
y_train=[]
y_dir= '/kaggle/input/assignment/Assignment/Train_Data/Binary_Maps'
for dirname, _, filenames in os.walk('/kaggle/input/assignment/Assignment/Train_Data/Originals'):
    for filename in filenames:
        x=cv2.imread(os.path.join(dirname, filename))
        x=cv2.resize(x, (512, 512),interpolation = cv2.INTER_NEAREST)
        y_file= filename[:-4]+'_torso.png'
        y=cv2.imread(os.path.join(y_dir,y_file))
        y=cv2.resize(y, (512, 512),interpolation = cv2.INTER_NEAREST)
        y = cv2.cvtColor(y, cv2.COLOR_BGR2GRAY)
        #y = cv2.merge((y,y,y))
        #y_mask = cv2.bitwise_and(x, x, mask=y)
        y_train.append(y)
        x_train.append(x)

In [3]:
x_train=np.array(x_train)
y_train=np.array(y_train)

In [4]:
!pip install segmentation_models

In [5]:
%env SM_FRAMEWORK=tf.keras

In [6]:
import segmentation_models as sm

BACKBONE = 'resnet50'
preprocess_input = sm.get_preprocessing(BACKBONE)

# load your data
#x_train, y_train, x_val, y_val = load_data(...)

# preprocess input
#x_train = preprocess_input(x_train)
#x_val = preprocess_input(x_val)

# define model
model = sm.Unet('resnet50', classes=1, activation='softmax')
model.compile(
    'Adam',
    loss=sm.losses.bce_jaccard_loss,
    metrics=[sm.metrics.iou_score],
)

# fit model
# if you use data generator use model.fit_generator(...) instead of model.fit(...)
# more about `fit_generator` here: https://keras.io/models/sequential/#fit_generator


In [ ]:
y_train=y_train.astype('float32')
tf.config.run_functions_eagerly(True)
model.fit(
   x=x_train,
   y=y_train,
   batch_size=1,
   epochs=100,
   #validation_data=(x_test, y_test),
)